In [293]:
import numpy as np
import pandas as pd
from datetime import datetime

In [294]:
rd_station_profile = pd.read_csv("../data/rd_station_profile")
info_student_unimestre = pd.read_csv("../data/info_student_unimestre.csv")
first_cd_turma_student = pd.read_csv("../data/first_cd_turma_for_student_unimestre.csv")
dt_cadastro_student = pd.read_csv("../data/dt_cadastro_student_unimestre.csv")

### Checking the data frames


In [295]:
rd_station_profile.shape

(41340, 32)

In [296]:
info_student_unimestre.shape

(12056, 5)

In [297]:
info_student_unimestre.sample()

,cd_pessoa,ds_email,ds_cidade,dt_nascimento,min_dt_log
9365,160003450,suzanapsc@gmail.com,Joinville,1978-04-03 00:00:00,NaN


In [298]:
info_student_unimestre.isna().sum()

cd_pessoa           0
ds_email            0
ds_cidade        2248
dt_nascimento    1156
min_dt_log       8144
dtype: int64

In [299]:
first_cd_turma_student.shape

(15901, 5)

In [300]:
first_cd_turma_student.isna().sum()

cd_pessoa       0
cd_turma        0
ano          8630
semestre      517
min_date     9163
dtype: int64

In [301]:
dt_cadastro_student.isna().sum()

cd_pessoa         0
dt_cadastro    1265
dtype: int64

Trying to get less null values on the min_dt_log

In [302]:
info_student_unimestre.loc[:, 'min_dt_log'] = (
    info_student_unimestre['min_dt_log']
    .fillna(first_cd_turma_student['min_date'])
    .fillna(dt_cadastro_student['dt_cadastro'])
)

In [303]:
info_student_unimestre.isna().sum()

cd_pessoa           0
ds_email            0
ds_cidade        2248
dt_nascimento    1156
min_dt_log        209
dtype: int64

With this I was able to reduce 
- *min_dt_log* had 9163 null values now 209

### Checking how many rows got in a inner join (getting just the intersection)

In [304]:
merge_inner = rd_station_profile.merge(info_student_unimestre, left_on='email', right_on='ds_email', how='inner')

In [305]:
merge_inner.shape

(4150, 37)

### Left Join

In [306]:
merge_left = rd_station_profile.merge(info_student_unimestre, left_on='email', right_on='ds_email', how='left')

In [307]:
merge_left.shape # 41428 got more rows than the origial

(41428, 37)

Got more rows than the left data frame, so will check for duplicates values 

In [308]:
rd_station_profile.groupby("email")["email"].count()

email
0137.marcogiovanella@cnec.br     1
0337.mauro@bradesco.com.br       1
0337.william@bradesco.com.br     1
0358.camila@bradesco.com.br      1
0358.gerencia@bradesco.com.br    1
                                ..
zullo_gui@hotmail.com            1
zulmafernandesstolf@gmail.com    1
zulmalira.contabil@gmail.com     1
zwessel@hotmail.com              1
zyon.beto@gmail.com              1
Name: email, Length: 41340, dtype: int64

In [309]:
info_student_unimestre.groupby("ds_email")["ds_email"].count()

ds_email
 leory@r19.com.br                     1
 renatomontag@terra.com.br            1
00000000/00@sememail.com             27
00000000000@sememail.com            210
00000000001@sememail.com              1
                                   ... 
znf.nem@gmail.com                     1
zpierdona@prsp.mpf.gov.br             1
zuleikaromao@gomesdacosta.com.br      1
zulmarpr@gmail.com                    1
zvirtes@joinville.udesc.br            1
Name: ds_email, Length: 10520, dtype: int64

In [310]:
info_student_unimestre.groupby("cd_pessoa")["cd_pessoa"].count()

cd_pessoa
1001         1
1002         1
100003       1
100011       1
100046       1
            ..
170000461    1
170000462    1
170000463    1
170000464    1
170000465    1
Name: cd_pessoa, Length: 12056, dtype: int64

Creating a df to be only one row per ds_email

In [311]:
info_student_unimestre_unique = info_student_unimestre.groupby('ds_email').first().reset_index()

In [312]:
info_student_unimestre_unique.shape

(10520, 5)

### Doing the join again with the new df

In [313]:
merge_left = rd_station_profile.merge(info_student_unimestre_unique, left_on='email', right_on='ds_email', how='left')

In [314]:
merge_left.shape

(41340, 37)

In [315]:
merge_left = merge_left.drop(columns='Unnamed: 0')

In [316]:
merge_left.columns

Index(['email', 'lead_scoring_-_perfil', 'url_pública', 'estágio_no_funil',
       'total_de_conversões', 'lead_scoring_-_interesse',
       'status_para_comunicação_por_email', 'data_da_primeira_conversão',
       'eventos_(últimos_100)', 'origem_da_última_conversão',
       'data_da_última_conversão', 'origem_da_primeira_conversão', 'empresa',
       'tags', 'data_da_última_oportunidade', 'estado',
       'valor_total_da_oportunidade_no_crm_(última_atualização)',
       'qualificação_da_oportunidade_no_crm_(última_atualização)',
       'etapa_do_funil_de_vendas_no_crm_(última_atualização)',
       'nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)',
       'origem_da_oportunidade_no_crm_(última_atualização)',
       'possui_interesse_em_realizar_uma_pós-graduação_ou_mba?',
       'data_de_nascimento', 'desejo_receber_o_sustentare_news',
       'como_você_conheceu_a_sustentare:', 'universidade',
       'qual_o_curso_de_interesse?', 'cidade_final', 'cargo_final',
      

Creating a new column saying it's a student when a cd_pessoa is found

In [317]:
merge_left['aluno'] = (merge_left['cd_pessoa'].notnull()).astype(int)
merge_left['aluno'].value_counts()


aluno
0    37278
1     4062
Name: count, dtype: int64

## Got different columns with representing the same thing, compiling all in one new columns

Represents city of the person: 'ds_cidade', 'Cidade Final'

In [318]:
merge_left.loc[:, 'cidade'] = (
    merge_left['ds_cidade']
    .fillna(merge_left['cidade_final'])
)

Represents city of the person: 'ds_cidade', 'Cidade Final'

In [319]:
merge_left.loc[:, 'data_nascimento'] = (
    merge_left['dt_nascimento']
    .fillna(merge_left['data_de_nascimento'])
)

In [320]:
merge_left.isna().sum()

email                                                                    0
lead_scoring_-_perfil                                                    0
url_pública                                                              0
estágio_no_funil                                                         0
total_de_conversões                                                      0
lead_scoring_-_interesse                                                 0
status_para_comunicação_por_email                                        0
data_da_primeira_conversão                                               0
eventos_(últimos_100)                                                    0
origem_da_última_conversão                                               0
data_da_última_conversão                                                 0
origem_da_primeira_conversão                                             0
empresa                                                              23786
tags                     

With this I was able to reduce 
- *Cidade* had 35856 null values now 33986
- *Data Nascimento* had 38346 null values now 36117

Dropping columns with replicate values

In [321]:
merge_left = merge_left.drop(columns=['ds_cidade', 'cidade_final', 'dt_nascimento', 'data_de_nascimento'])

All not null

In [322]:
merge_left[merge_left.notna().all(axis=1)]

,email,lead_scoring_-_perfil,url_pública,estágio_no_funil,total_de_conversões,lead_scoring_-_interesse,status_para_comunicação_por_email,data_da_primeira_conversão,eventos_(últimos_100),origem_da_última_conversão,...,qual_o_curso_de_interesse?,cargo_final,area_atuacao,interesse_final,ds_email,cd_pessoa,min_dt_log,aluno,cidade,data_nascimento
5431,carolineklock@hotmail.com,c,http://app.rdstation.com.br/leads/public/8717b...,Lead Qualificado,16,120,True,2015-09-01 19:17:37 -0300,jornada-assessment-instrumentos-para-lideranca...,Busca Orgânica | Bing,...,Educação empresarial / Remuneração,Outros Cargos,RH,formulario-de-pre-inscricao,carolineklock@hotmail.com,128609.0,2018-04-05 19:53:30,1,Joinville,1985-09-21 00:00:00
15881,stella.bousfield@gmail.com,c,http://app.rdstation.com.br/leads/public/1b271...,Cliente,27,170,True,2015-12-08 10:18:55 -0200,webinar-storytelling / indicado-sarau-de-negoc...,Desconhecido,...,Responsabilidade social,Outros Cargos,Marketing,webinar-storytelling,stella.bousfield@gmail.com,160000017.0,2020-02-27 17:28:20,1,Joinville,1977-06-02 00:00:00


In [323]:
merge_left.columns

Index(['email', 'lead_scoring_-_perfil', 'url_pública', 'estágio_no_funil',
       'total_de_conversões', 'lead_scoring_-_interesse',
       'status_para_comunicação_por_email', 'data_da_primeira_conversão',
       'eventos_(últimos_100)', 'origem_da_última_conversão',
       'data_da_última_conversão', 'origem_da_primeira_conversão', 'empresa',
       'tags', 'data_da_última_oportunidade', 'estado',
       'valor_total_da_oportunidade_no_crm_(última_atualização)',
       'qualificação_da_oportunidade_no_crm_(última_atualização)',
       'etapa_do_funil_de_vendas_no_crm_(última_atualização)',
       'nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)',
       'origem_da_oportunidade_no_crm_(última_atualização)',
       'possui_interesse_em_realizar_uma_pós-graduação_ou_mba?',
       'desejo_receber_o_sustentare_news', 'como_você_conheceu_a_sustentare:',
       'universidade', 'qual_o_curso_de_interesse?', 'cargo_final',
       'area_atuacao', 'interesse_final', 'ds_email

### Change dtype for columns that are date

In [282]:
# Assuming 'df' is your DataFrame and 'data_da_primeira_conversao' is your datetime column
merge_left['data_da_primeira_conversão'] = pd.to_datetime(merge_left['data_da_primeira_conversão'])
# Now format the datetime column to keep only the date part
merge_left['data_da_primeira_conversão'] = merge_left['data_da_primeira_conversão'].dt.strftime('%Y-%m-%d')
merge_left

C:\Users\anafl\AppData\Local\Temp\ipykernel_45952\2096275456.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  merge_left['data_da_primeira_conversão'] = pd.to_datetime(merge_left['data_da_primeira_conversão'])


AttributeError: Can only use .dt accessor with datetimelike values

Just changing - Work but can't use after to do one - other


# Final of these

In [328]:
def convert_columns_to_datetime(df, date_columns):
    """
    Convert specified columns in a DataFrame to datetime format.

    Parameters:
    - df: DataFrame
        The input DataFrame.
    - date_columns: list
        List of column names to convert to datetime.

    Returns:
    - DataFrame
        The DataFrame with specified columns converted to datetime.
    """
    df[date_columns] = df[date_columns].apply(lambda col: pd.to_datetime(col, errors='coerce'))
    for i in date_columns:
        df[i] = pd.to_datetime(df[i], utc=True).dt.tz_convert(None)
    return df

# Example usage:
date_columns = ['data_da_primeira_conversão', 'data_da_última_conversão', 'data_da_última_oportunidade', 'data_nascimento', 'min_dt_log']
merge_left = convert_columns_to_datetime(merge_left, date_columns)
merge_left.dtypes


email                                                                        object
lead_scoring_-_perfil                                                        object
url_pública                                                                  object
estágio_no_funil                                                             object
total_de_conversões                                                           int64
lead_scoring_-_interesse                                                      int64
status_para_comunicação_por_email                                              bool
data_da_primeira_conversão                                           datetime64[ns]
eventos_(últimos_100)                                                        object
origem_da_última_conversão                                                   object
data_da_última_conversão                                             datetime64[ns]
origem_da_primeira_conversão                                                

Doing separated all - 

In [329]:
def calculate_months_since_conversion(df):
    """
    Calculates the number of months since the first conversion based on the 'aluno' column.

    Parameters:
    - The input DataFrame containing the necessary columns.

    Returns:
    - A new column representing the number of months since the first conversion.
    """

    # If 'aluno' is 1, calculate the difference between 'min_dt_log' and 'data_da_primeira_conversão'
    # If not calculate the difference between today and 'data_da_primeira_conversão'
    months_since_conversion = np.where(
        df['aluno'] == 1,
        (df['min_dt_log'] - df['data_da_primeira_conversão']).dt.days / 30,
        (datetime.now() - df['data_da_primeira_conversão']).dt.days / 30
    )

    return months_since_conversion

merge_left['months_since_conversion'] = calculate_months_since_conversion(merge_left)
merge_left.sample()

,email,lead_scoring_-_perfil,url_pública,estágio_no_funil,total_de_conversões,lead_scoring_-_interesse,status_para_comunicação_por_email,data_da_primeira_conversão,eventos_(últimos_100),origem_da_última_conversão,...,cargo_final,area_atuacao,interesse_final,ds_email,cd_pessoa,min_dt_log,aluno,cidade,data_nascimento,months_since_conversion
28744,geovane@tupy.com.br,d,http://app.rdstation.com.br/leads/public/d23c3...,Lead,1,0,True,2017-05-08 22:55:46,Interessado: Gestão Tributária e Compliance,Referência | sustentare.net/site/curso/planeja...,...,NaN,NaN,Interessado: Gestão Tributária e Compliance,NaN,NaN,NaT,0,NaN,NaT,79.9


In [185]:
merge_left['aluno'].value_counts()

aluno
0    37278
1     3297
Name: count, dtype: int64

In [186]:
# Count the number of null values in 'min_dt_log' when 'aluno' is equal to 1
merge_left.loc[merge_left['aluno'] == 1, 'min_dt_log'].isnull().sum()

0

In [187]:
def drop_rows_when_student_and_min_dt_log_is_null(df):
    """
    Drop rows from a DataFrame based on specified conditions.

    Parameters:
    - df: DataFrame
        The input DataFrame.

    Returns:
    - DataFrame
        The DataFrame with rows dropped based on the specified conditions.
    """
    # Drop rows based on conditions
    df = df.drop(df[(df['aluno'] == 1) & df['min_dt_log'].isnull()].index)

    return df

merge_left = drop_rows_when_student_and_min_dt_log_is_null(merge_left)

# Verify the changes
merge_left.loc[merge_left['aluno'] == 1, 'min_dt_log'].isnull().sum()


0

In [188]:
merge_left['aluno'].value_counts()

aluno
0    37278
1     3297
Name: count, dtype: int64